In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3 import A2C

from gym.wrappers import GrayScaleObservation

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
import os
from stable_baselines3.common.callbacks import BaseCallback

import optuna

from stable_baselines3.common.evaluation import evaluate_policy

import os


env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)


In [2]:
log_dir = './OptunaParamterAdjust_Log'
os.makedirs(log_dir, exist_ok=True)

env = Monitor(env, log_dir)

env = GrayScaleObservation(env,keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4,channels_order='last')



In [3]:
def optimize_A2C(trial):
    return {
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
    }

In [4]:
def optimize_agent(trial):

    try:
        env = gym_super_mario_bros.make('SuperMarioBros-v0')
        env = JoypadSpace(env, SIMPLE_MOVEMENT)

        log_dir = './OptunaParamterAdjust_Log'
        os.makedirs(log_dir, exist_ok=True)

        env = Monitor(env, log_dir)

        env = GrayScaleObservation(env,keep_dim=True)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env,4,channels_order='last')


        model_params = optimize_A2C(trial)


        tensorboard_log = r'./OptunaParameterAdjust_tensorboard_log'
        model = A2C("CnnPolicy", env, verbose=0,tensorboard_log=tensorboard_log,**model_params)
        # model.learn(total_timesteps=1000)
        model.learn(total_timesteps=200000)

        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)


        env.close()

        OPT_DIR  = r'./OPT_A2C'
        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        return -1000


In [5]:
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=100)

[I 2022-06-24 14:06:28,238] A new study created in memory with name: no-name-4a396984-93b8-4380-9284-c6b536ac086a
C:\Users\admin\anaconda3\envs\openai\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning:

overflow encountered in ubyte_scalars

[I 2022-06-24 14:46:11,911] Trial 0 finished with value: 737.0 and parameters: {'gamma': 0.9941812152746141, 'learning_rate': 4.5225017679358453e-05}. Best is trial 0 with value: 737.0.
C:\Users\admin\anaconda3\envs\openai\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning:

overflow encountered in ubyte_scalars

[I 2022-06-24 15:25:02,802] Trial 1 finished with value: 735.0 and parameters: {'gamma': 0.8486982210765408, 'learning_rate': 1.7053074143340685e-05}. Best is trial 0 with value: 737.0.
C:\Users\admin\anaconda3\envs\openai\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning:

overflow encountered in ubyte_scalars

[I 2022-06-24 16:03:59,867] Trial 2 finished with value: 732.0 and par

KeyboardInterrupt: 

In [6]:
dir(study)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ask',
 '_is_multi_objective',
 '_log_completed_trial',
 '_optimize_lock',
 '_pop_waiting_trial_id',
 '_stop_flag',
 '_storage',
 '_study_id',
 '_tell',
 'add_trial',
 'add_trials',
 'ask',
 'best_params',
 'best_trial',
 'best_trials',
 'best_value',
 'direction',
 'directions',
 'enqueue_trial',
 'get_trials',
 'optimize',
 'pruner',
 'sampler',
 'set_system_attr',
 'set_user_attr',
 'stop',
 'study_name',
 'system_attrs',
 'tell',
 'trials',
 'trials_dataframe',
 'user_attrs']

In [7]:
study.best_params

{'gamma': 0.9357882315009981, 'learning_rate': 3.0976565201136994e-05}

In [8]:
study.best_trial

FrozenTrial(number=3, values=[741.0], datetime_start=datetime.datetime(2022, 6, 24, 16, 3, 59, 869644), datetime_complete=datetime.datetime(2022, 6, 24, 16, 42, 54, 737956), params={'gamma': 0.9357882315009981, 'learning_rate': 3.0976565201136994e-05}, distributions={'gamma': LogUniformDistribution(high=0.9999, low=0.8), 'learning_rate': LogUniformDistribution(high=0.0001, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None)